In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from feature_extraction import Blob, Words
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from time import time
from pprint import pprint
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from gp_test import main
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
data = main(sub='python', start='01/17/2018', end='01/24/2018')
data.title = data.title.astype('str')
#data = next(data_gen)
#data = pd.read_csv('processed_posts.csv.bz2')
#data = pd.read_csv('processed_python.csv.bz2', encoding='ISO-8859-1', compression='bz2')
data = pd.read_csv('processed_datascience.csv.bz2')
data.drop_duplicates('title', inplace=True)
data = data[np.abs(data.ups-data.ups.mean())<=(2*data.ups.std())]
data['logups'] = np.log1p(data['ups'])

#data = data[data['subreddit'] == 'r/books']
train_X, test_X, train_y, test_y = train_test_split(data.title, 
                                                    data.logups, 
                                                    test_size=0.20,
                                                    random_state=42)

In [38]:
data.ups.mean()

7.3170731707317076

In [35]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
                      
            ('words', Words()),
                      
            ('title', Pipeline([
                ('tfidf', TfidfVectorizer(ngram_range=(1,3), sublinear_tf=True, stop_words='english')),
                ('svd', TruncatedSVD(n_components=120)),
                ('normalize', MinMaxScaler(copy=False)),
                ('selector', SelectPercentile(f_classif, percentile=10))
            ])),
            
            ('blob', Pipeline([
                ('all', Blob()),
                ('minmax', MinMaxScaler()),
            ])),
            
            ])),
    ('clf', Ridge()),
        ])

In [36]:
pipeline.fit(train_X, train_y)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('words', Words()), ('title', Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [37]:
y = pipeline.predict(test_X)
mean_absolute_error(y_pred=np.expm1(y), y_true=np.expm1(test_y))

5.630013960986278

In [40]:
index = 2
answer = pipeline.predict(pd.Series(['A Gentle Introduction to N-Dimensional Arrays in Python with NumPy - Machine Learning Mastery']))[0]
np.expm1(answer)

6.1857091209382684